In [1]:
import pandas as pd
import numpy as np
from functions import abbr_substitutor

df = pd.read_csv("E:/Data_exploration/GitHub/test-project-master/test-project-master/triplets/all_triplets_simplified_1M.csv", encoding='latin-1', names=['Patient_id', 'relation', 'object'])
df['Patient_id'] = df['Patient_id'].str.replace('Patiënt-', '')
df['Patient_id'] = pd.to_numeric(df['Patient_id'], errors='coerce')
df = df.dropna(subset=['Patient_id'])
df = df[np.isfinite(df['Patient_id'])]
df['Patient_id'] = df['Patient_id'].astype(int)
df['object'] = df['object'].astype(str)
mask = df['object'].str.contains('@')
# Drop the rows where the character is present
df = df[~mask]

In [2]:
# Group consecutive rows with the same patient ID
grouped = df.groupby((df['Patient_id'] != df['Patient_id'].shift()).cumsum())

# Create a new DataFrame with desired columns
new_df = pd.DataFrame({
    'index': grouped.apply(lambda x: x.index[0]),
    'Patient Id': grouped['Patient_id'].first(),
    'keywords': grouped.apply(lambda x: x.loc[x['relation'] != 'affiliatie',['object']].values.flatten().tolist())
}).reset_index(drop=True)

new_df['keywords'] = new_df['keywords'].apply(lambda x: list(set(x)))
new_df['keywords'] = new_df['keywords'].apply(lambda x: abbr_substitutor(' '.join(x)))
new_df['keywords']= new_df['keywords'].apply(lambda x: x.split())
new_df

,index,Patient Id,keywords
0,0,775871,[agenda]
1,3,234932,"[billen, omeprazol, jaszak, vooradig, cyste, r..."
2,147,774275,[jaqueline]
3,148,214290,"[vel, medicijnen, jeuk, diabetes]"
4,152,199439,[]
...,...,...,...
461732,2492989,1196418,"[mevrouw, lactulose, verstopt]"
461733,2492993,344495,"[pijnscore, pijnklachten]"
461734,2492996,1713310,"[herbeoordelen, vlaflip, medicatie, bladdersca..."
461735,2493011,1713316,[bovenlichaam]


In [3]:
from collections import Counter
import itertools

nested_column_values = new_df['keywords'].tolist()
column_values = list(itertools.chain.from_iterable(nested_column_values))

word_freq_list = Counter(column_values).most_common()
common_words = [word for word, freq in word_freq_list if freq > 5]

new_df['keywords'] = new_df['keywords'].apply(lambda words: [word for word in words if word in common_words])
new_df['Number of Objects'] = new_df['keywords'].apply(lambda x: len(x))

In [170]:
three_objects = new_df[(new_df['Number of Objects'] > 2) & (new_df['Number of Objects'] < 4)]
three_objects = three_objects.drop('index', axis=1)
three_objects.to_csv('keywords3_v1.csv', index=False)

In [171]:
pd.read_csv('keywords3_v1.csv')

,Patient Id,keywords,Number of Objects
0,775905,"['hallucinaties', 'hond', 'angst']",3
1,775935,"['paracetamol', 'dementie', 'wegraking']",3
2,773360,"['buik', 'ernstig', 'pijn']",3
3,190201,"['onderrug', 'been', 'pijn']",3
4,628958,"['verbandje', 'hand', 'pijn']",3
...,...,...,...
61031,228211,"['esomeprazol', 'pantoprazol', 'aktief']",3
61032,1195708,"['paracetamol', 'duizelig', 'plus']",3
61033,1027104,"['insuline', 'gdc', 'bloedsuikers']",3
61034,1713021,"['steunkous', 'kg.', 'aantrekkous']",3
